In [36]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(.3985 * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

# core_list = np.array([hex(2**n) for n in range(20)])

# channel2_core_list = [spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, spcm.SPCM_DDS_CORE20]

dds.cores_on_channel(1, spcm.SPCM_DDS_CORE8,spcm.SPCM_DDS_CORE9,spcm.SPCM_DDS_CORE10,spcm.SPCM_DDS_CORE11)
card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

2


In [12]:
dds.get_cores_on_channel(0)

1048575

In [13]:
spcm.SPCM_DDS_CORE8

256

In [2]:
print(int(spcm.SPCM_DDS_CORE8))

256


In [38]:
def compute_tweezer_1064_phases(n_tweezers,amplitudes):
    phase_tweezer_array = np.empty([n_tweezers])
    for tweezer_idx in range(n_tweezers):
        if tweezer_idx == 0:
            phase_tweezer_array[0] = 360
        else:
            phase_ij = 0
            for j in range(1,tweezer_idx):
                phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*amplitudes[tweezer_idx]
            phase_i = (phase_ij % 2*np.pi) * 360
            phase_tweezer_array[tweezer_idx] = phase_i
    return phase_tweezer_array

# horizontal array

f_list1 = np.linspace(72.25e6,74.e6,4)

n_tweezers1 = len(f_list1)

a_list1 = np.array([0.15,0.12,.105,.1])*2

f_list1 = np.array(f_list1)
a_list1 = np.array(a_list1)

phases1 = compute_tweezer_1064_phases(len(f_list1),a_list1)

# vertical array
rng = .95e6
f_list2 = 75.e6 + np.linspace(-rng,rng,4)

n_tweezers2 = len(f_list2)

a_list2 = np.array([.26,.23,.23,.26])

f_list2 = np.array(f_list2)
a_list2 = np.array(a_list2)

phases2 = compute_tweezer_1064_phases(len(f_list2),a_list2)

# def normalize_alist(alist):
#     sum = np.sum(alist)
#     return alist/sum
# a_list = normalize_alist(a_list)

for tweezer_idx in range(len(f_list1)):
    if tweezer_idx < len(f_list1):
        dds[tweezer_idx].amp(a_list1[tweezer_idx])
        dds[tweezer_idx].freq(f_list1[tweezer_idx])
        dds[tweezer_idx].phase(phases1[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)

for tweezer_idx in range(len(f_list2)):
    if tweezer_idx < len(f_list2):
        tweezer_idx = tweezer_idx + 8
        dds[tweezer_idx].amp(a_list2[tweezer_idx-8])
        dds[tweezer_idx].freq(f_list2[tweezer_idx-8])
        dds[tweezer_idx].phase(phases2[tweezer_idx-8])
    else:
        dds[tweezer_idx].amp(0.)
    
dds.exec_at_trg()
dds.write_to_card()
trigger.force()

# Start command including enable of trigger engine

# input("Press Enter to Exit")

In [39]:
# card.close(card._handle)
card.stop()
card.close(card._handle)